<div class="alert alert-block alert-danger">
<b>Warning:</b> this notebook is still a work in progress!
</div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Installation" data-toc-modified-id="Installation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Installation</a></span></li><li><span><a href="#Tutorial" data-toc-modified-id="Tutorial-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Tutorial</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Explore-the-data" data-toc-modified-id="Explore-the-data-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Explore the data</a></span></li></ul></li></ul></div>

# 2D-SXDM tutorial

This notebook introduces basic exploration and preprocessing of 2D-SXDM data obatined on beamline ID01 @ ESRF using the in-house developed library `sxdm`. 

**Requirements**:
* An ESRF single sign-on (SSO)
* Basic knowledge of [Jupyter notebooks](https://jupyter.org/)

## Installation

This notebook is meant to be run on the [Jupyter-SLURM hub](https://jupyter-slurm.esrf.fr/). Instructions on how to achieve this are given in the [README](https://gitlab.esrf.fr/id01-science/id01-sxdm-utils/-/blob/main/README.md) of the `sxdm` repository (requires SSO for access). You should be reading this from within jupyter-slurm!

To run the notebook a few python dependecies are necessary. To simplify things, these are stored in a virtual environment accessible from jupyter-slurm and maintained by the ID01 staff. To install the virtual environment run the cell below:

In [1]:
%%bash

pip3 install ipykernel
source /data/id01/inhouse/data_analysis/software/pyenvs/sxdm.slurm/bin/activate
python3 -m ipykernel install --user --name sxdm.slurm

Installed kernelspec sxdm.slurm in /home/esrf/zatterin/.local/share/jupyter/kernels/sxdm.slurm


Now refresh the webpage. One this is done, change the active kernel by using the menu bar on the top and clicking on `Kernel > Change kernel > sxdm.slurm`. You are now ready to run the notebook!

## Tutorial

<div class="alert alert-block alert-warning">
<b>Warning:</b> This notebook deals with SXDM data produced by BLISS, the new control system of ESRF. BLISS is still under development and its structure changes relatively often. Some code in this notebook may therefore be out of date (as well as some info/explanations) and not work correctly. If this is the case, please submit an <a href="https://gitlab.esrf.fr/id01-science/id01-sxdm-utils/-/issues">issue</a>
.
</div>

### Introduction

2D-SXDM refers to SXDM data collected at a single angle, i.e. without rocking the sample. This corresponds to the output of individual `asxdm()` or `dsxdm()` commands run in `BLISS` on ID01.

During an experiment data is saved on the ESRF NICE cluster. Each experiment has a directory tree dedicated to it. For example, the inhouse `id012203` experiment is located in  `/data/id01/inhouse/data/IHR/id012203/id01/`. A `m21005b` sub-folder is present in the main folder:

In [1]:
!tree -L 2 /data/id01/inhouse/data/IHR/id012203/id01/

/data/id01/inhouse/data/IHR/id012203/id01/
├── analysis
│   ├── coms.npy
│   ├── data
│   ├── _debug_ipympl.ipynb
│   ├── export
│   ├── id01-sxdm-utils
│   ├── m21005b_103
│   ├── m21005b_local-RSMs.h5
│   ├── m21005b_RSMs.h5
│   ├── roi_int.npy
│   ├── scans-bliss-MH.ipynb
│   ├── scans_sxdm-bliss_m21005b.ipynb
│   ├── SIMPLE_sxdm_analyse_m21005b_103_EZ.ipynb
│   ├── SIMPLE_sxdm_analyse_m21005b_103.ipynb
│   ├── sxdm_analyse_m21005b_103_explore-roi.ipynb
│   ├── sxdm_analyse_m21005b_103_iptilt-roi.ipynb
│   └── sxdm_merge_m21005b.ipynb
├── id012203_id01.h5
├── m21005b
│   ├── id012203_m21005b.h5
│   ├── m21005b_0001
│   ├── m21005b_0002
│   ├── m21005b_0003
│   ├── m21005b_0004
│   ├── m21005b_0005
│   ├── m21005b_0006
│   ├── m21005b_0007
│   ├── m21005b_0008
│   ├── m21005b_0009
│   ├── m21005b_0010
│   ├── m21005b_0011
│   ├── m21005b_align
│   ├── m21005b_align_sxdm
│   └── m21005b_herc22_morning
└── sample
    ├── id012203_sample.h5
    ├── sample_0001
    ├── sample_0002
    ├─

The `m21005b` folder contains all the data and metadata pertaining the sample named `m21005b`. It was created by using the `newsample` BLISS command. One should thus run this command every time a sample is changed. 

Within `m21005b`, one finds several `m21005b_xxxx` folders. Each folder corresponds to a dataset, and was initialised using the `newdataset` command. Again the idea is to isolate conceptually different scans routines from eachother. For example, two datasets may be two sets of SXDM scans performed at different temperatures.

Finally, within each dataset directory there will be a folder for each scan performed. For example, taking dataset `sample_0001`:

In [2]:
!tree -L 3 /data/id01/inhouse/data/IHR/id012203/id01/m21005b/m21005b_0001

/data/id01/inhouse/data/IHR/id012203/id01/m21005b/m21005b_0001
├── id012203-m21005b-0001.h5
├── m21005b_0001.h5
├── scan0001
│   └── mpx1x4_0000.h5
├── scan0002
│   └── mpx1x4_0000.h5
├── scan0003
│   └── mpx1x4_0000.h5
├── scan0004
│   └── mpx1x4_0000.h5
└── scan0005
    └── mpx1x4_0000.h5

5 directories, 7 files


Everytime one enters one of these subdirectories, an [HDF5](https://www.hdfgroup.org/solutions/hdf5/) (`.h5`) file is present. Indeed there is one even at the very top of the directory tree, above the `m21005b` directory:

In [3]:
%ls -lh /data/id01/inhouse/data/IHR/id012203/id01/

total 50K
drwxrwxrwx  8 opid01 id01 4,0K juil. 21 14:11 analysis/
-rw-rw-r--  1 opid01 id01  43K mars  29 20:50 id012203_id01.h5
drwxrwxr-x 16 opid01 id01 4,0K mars   9 22:21 m21005b/
drwxrwxr-x  8 opid01 id01 4,0K mars  15 15:20 sample/


The HDF5 format allows to store data and metadata together in a single file. Moreover, a given `.h5` file can contain links to another `.h5` file. Thus, even if the "real" data is contained in an `.h5` file somwhere down a directory tree, a "master" `.h5` file can point to it. This is exactly what happens in the present case, where each file in a parent directory contains links to the file in the directory below it. 

Most useful is the file that pertains to a certain dataset,

In [4]:
path_dset = '/data/id01/inhouse/data/IHR/id012203/id01/m21005b/m21005b_0001/m21005b_0001.h5'

To quickly inspect its contents one can use a tool developed at the EUXFEL that traverses the file's contents and exposes them as an html (interactive) output:

In [5]:
from h5glance import H5Glance as hp5 

h5p(path_dset)

NameError: name 'h5p' is not defined

The contents of the file follow the [NeXus])https://www.nexusformat.org/) convention - eventually at any synchrotron in the world the data files will have this same structure. The meaning of the data (techicnally: a `dataset`) contained in each hdf5 "folder" (technically: a `group`) should be easy to understand by exploring the file structure (If not, check the link above for more info). The ESRF software group has developed a [series of tools](http://www.silx.org/doc/silx/latest/Tutorials/io.html) to easily visualise these hierarchical data when sitting at the beamline.

### Import libraries

In [ ]:
# %matplotlib widget

# import numpy as np
# import matplotlib.pyplot as plt
# import sxdm

In [1]:
%matplotlib widget

# from pathlib import Path
# import sys

# sys.path.insert(0,'{}/repos/id01-sxdm-utils/'.format(Path.home()))

import sxdm
import numpy as np
import h5py

import matplotlib.pyplot as plt
import ipywidgets as ipw

from datetime import datetime
from functools import wraps

def ioh5(func):
    @wraps(func) # to get docstring of dectorated func
    def wrapper(filename, *args, **kwargs):
        with h5py.File(filename, 'r') as h5f:
            return func(h5f, *args, **kwargs)
    return wrapper

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


### Explore the data

In [115]:
path_exp = '/data/id01/inhouse/data/IHR/id012203/id01'

In [116]:
name_dset = '0008'
path_dset = f'{path_exp}/m21005b/m21005b_{name_dset}/m21005b_{name_dset}.h5'

In [117]:
scan_nos = []
with h5py.File(path_dset, 'r') as h5f:
    _nscans = len(list(h5f.keys()))
    for idx in range(1, _nscans+1):
        _scan = f'{idx}.1'
        try:
            _command = h5f[_scan]['title'][()].decode()
            _datetime = h5f[f'{_scan}/start_time'][()].decode()
            _datetime = datetime.fromisoformat(_datetime).strftime('%b %d | %H:%M:%S')
            _day = _datetime.split(' ')[1]
        except KeyError:
            pass
        if 'sxdm' in _command:
            print(_scan, _command)
            scan_nos.append(_scan)

1.1 asxdm( pix, 47.0, 53.0, 100, piy, 40.0, 46.0, 100, 0.01 )
2.1 asxdm( pix, 48.0, 54.0, 60, piy, 41.0, 47.0, 60, 0.01 )


In [122]:
def get_roi(scan_idx, roi_name):
    with h5py.File(path_dset, 'r') as h5f:
        _entry = scan_nos[scan_idx]
        _sh = [h5f[_entry][f'technique/{x}'][()] for x in ('dim0', 'dim1')]
        data = h5f[_entry][f'measurement/mpx1x4_{roi_name}'][()].reshape(*_sh)
    
    return data

In [123]:
@ioh5
def get_positioner(h5fname, scan_idx, motor_name):
    entry = scan_nos[scan_idx]
    mot = h5fname[f'/{entry}/instrument/positioners/{motor_name}'][()]
    
    return mot

In [124]:
%matplotlib widget
%matplotlib widget

fig, ax = plt.subplots(1,1, figsize=(4,4), layout='tight')

dmap = ax.imshow(get_roi(0, 'roi1'))

@ipw.interact(i=(0, len(scan_nos)-1, 1))
def pltdmap(i=0):
    _map = get_roi(i, 'roi1')
        
    dmap.set_data(_map)
    dmap.set_clim(_map.min(), _map.max())
    
    fzpx = get_positioner(path_dset, i, 'fzpx')
    ax.set_title(f'{fzpx:.0f}')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='i', max=1), Output()), _dom_classes=('widget-interact',)…